In [16]:
import pandas as pd
import numpy as np
import math
import lasagne
import theano
import theano.tensor as T
import time
from tqdm import tqdm
import json

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import hinge_loss
from sklearn.decomposition import PCA

/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
new_data = pd.read_csv("githubUsersFull.csv")

In [19]:
all_data = pd.read_csv('hhImprovedCleared.csv')

In [21]:
X = all_data.drop("Salary",axis = 1).as_matrix()
y = np.log(all_data['Salary'].values)

In [22]:
new_data = pd.read_csv("githubUsersFull.csv")
c = pd.read_csv('sending_value.csv')
for i in new_data.columns:
    if i not in c['data'].values:
        new_data[i] = np.zeros(len(new_data))

In [23]:
a = PCA(n_components=3)
X = a.fit_transform(X)
ans = a.transform(new_data)

In [28]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.1)

In [29]:
z = SVR().fit(Xtrain,Ytrain)
answer = z.predict(ans)
answer = pd.DataFrame(answer,columns = ['Salay'])
answer.to_csv('Salay.csv')

In [42]:
!ls -lht pca.pickle

-rw-r--r-- 1 root root 1.9K Dec 11 14:17 pca.pickle


In [40]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(z, f)
with open('pca.pickle', 'wb') as f:
    pickle.dump(a, f)

In [ ]:
input_x = T.matrix('one of user',dtype = 'float32')
target_y = T.matrix('answer',dtype = 'float32')

l_in = lasagne.layers.InputLayer(shape= (None, 3), input_var = input_x)

dense = lasagne.layers.DenseLayer(
            l_in, num_units=50,
            nonlinearity=lasagne.nonlinearities.rectify,
W=lasagne.init.GlorotUniform())
dense = lasagne.layers.DenseLayer(
            l_in, num_units=20,
            nonlinearity=lasagne.nonlinearities.rectify,
W=lasagne.init.GlorotUniform())
out = lasagne.layers.DenseLayer(dense,num_units=1,nonlinearity = None)

In [ ]:
predict = lasagne.layers.get_output(out)
loss = lasagne.objectives.squared_error(predict,target_y).mean()
weight = lasagne.layers.get_all_params(out,trainable=True)
updates = lasagne.updates.adam(loss,weight,learning_rate = 0.01)
train_fun = theano.function([input_x,target_y],loss,updates = updates,name='high')
accuarcy_fun = theano.function([input_x,target_y],loss)
predict_fun = theano.function([input_x],predict)
test = []
train = []

In [ ]:
predict.eval({input_x: X[:10].values.astype(np.float32)}).shape

In [ ]:
loss.eval({input_x: X[:10].values.astype(np.float32), target_y: y[:10].reshape((-1, 1)).astype(np.float32)})

In [ ]:
def iterate_minibatches(inputs, targets, batchsize):
    assert len(inputs) == len(targets)
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield inputs[excerpt], targets[excerpt]

In [ ]:
num_epochs = 100 #количество проходов по данным
train_r = []
val_r = []

batch_size = 50 #размер мини-батча

for epoch in tqdm(range(num_epochs)):
        # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(Xtrain, Ytrain,batch_size):
        inputs, target = batch
        train_err_batch= train_fun(inputs.astype("float32"), target.reshape((-1,1)).astype('float32'))
        train_err += train_err_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(Xtest, Ytest, batch_size):
        inputs, target = batch
        val_acc += accuarcy_fun(inputs.astype("float32"), target.reshape((-1,1)).astype('float32'))
    
    
#     # Then we print the results for this epoch:
#     print("Epoch {} of {} took {:.3f}s".format(
#         epoch + 1, num_epochs, time.time() - start_time))
    train_r.append(train_err/train_batches)
    val_r.append(np.mean(val_acc))
#    print("  training loss (in-iteration):\t\t{:.6f}".format(np.mean(train_err/train_batches)))
#     print("  train accuracy:\t\t{:.2f} %".format(
#         train_acc / train_batches * 100))
#     train.append(train_acc / train_batches * 100)
#    print("  validation loss:\t\t{:.2f}".format(
#        np.mean(val_acc)))


In [ ]:
from matplotlib import pyplot as plt
plt.plot(val_r)
plt.show()
plt.plot(train_r)
plt.show()